<a href="https://colab.research.google.com/github/mehdii190/neural-network/blob/main/src/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


num_epochs = 30
batch_size = 100
learning_rate = 0.001




transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])


train_dataset = torchvision.datasets.CIFAR10(root = "../../data/",
                                             train = True,
                                             transform = transform,
                                             download = True)


test_dataset = torchvision.datasets.CIFAR10(root = "../../data/",
                                             train = False,
                                             transform = transforms.ToTensor())


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)



100%|██████████| 170498071/170498071 [00:02<00:00, 80406839.52it/s]


Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [9]:
def conv3x3(in_channles, out_channles, stride = 1):
  return nn.Conv2d(in_channles, out_channles, kernel_size= 3 , stride= stride, padding= 1 ,bias = False)


class ResidualBlock(nn.Module):
  def __init__(self, in_channles, out_channles, stride = 1 , downsampling = None):
    super(ResidualBlock, self).__init__()
    self.conv1 = conv3x3(in_channles, out_channles, stride)
    self.bn1 = nn.BatchNorm2d(out_channles)
    self.relu = nn.ReLU(inplace= True)
    self.conv2 = conv3x3(in_channles, out_channles)
    self.bn2 = nn.BatchNorm2d(out_channles)
    self.downsampling = downsampling
  def forward(self,x):
    residual = x.clone()
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    if self.downsampling:
      residual = self.downsampling(x)
    out+= residual
    out = self.relu(out)
    return out